# Docker Swarm with Blinkt!

## Init Docker Swarm on Cluster

Zunächst die IP-Adresse feststellen.

In [ ]:
getent hosts $(hostname)

Geht nicht, da hier nur die Einträge aus der `hosts`-Datei ausgelesen werden.

In [ ]:
cat /etc/hosts

Besser die IP und Namen durch den DNS aus dem lokalen Netz verwenden.

Hier wird das Paket dnsutils nachinstalliert, falls nslookup nicht vorhanden ist.

In [ ]:
nslookup $(hostname) || (sudo apt-get install --yes dnsutils && nslookup $(hostname))

In [ ]:
FQDN=$(nslookup $(hostname) | grep Name | awk '{ print $2 }')
echo $FQDN

In [ ]:
IP=$(getent hosts ${FQDN} | awk '{ print $1 }')
echo $IP

## FABLAB NBG

In [ ]:
IP=172.16.20.26

In [ ]:
DOMAIN=$(getent hosts ${FQDN} | awk '{ print $2 }' | cut -d '.' -f2,3)
echo $DOMAIN

In [ ]:
docker node ls || docker swarm init --advertise-addr ${IP}

In [ ]:
JOIN_TOKEN=$(docker swarm join-token -q worker)
echo $JOIN_TOKEN

In [ ]:
# Setup ssh-key for user if not exists
ls ~/.ssh/id_rsa || ansible -i localhost, -m shell -a 'ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa' --connection=local localhost

Den SSH Zugang vorbereiten. Zunächst aus `knwon_hosts` löschen. Dann wieder bekannt geben und den ssh-key übertragen.

In [ ]:
for i in 01 02 03 04; do
    # Remove host key for ip
    CL_HOSTNAME=cluster-${i}
    echo ${CL_HOSTNAME}
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster FQDN
    ssh-keygen -R ${CL_HOSTNAME}.${DOMAIN}
    # IP Cluster 
    ssh-keygen -R $(getent hosts ${CL_HOSTNAME} | awk '{ print $1 }')
    # Add ip into known_hosts
    ssh-keyscan -H cluster-${i} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@cluster-${i}
done

In [ ]:
for i in 21.76 23.27 23.237 23.242; do
    PRE_FIX=172.16.
    # Remove host key for ip
    CL_HOSTNAME=${PRE_FIX}${i}
    echo ${CL_HOSTNAME}
    # Cluster Hostname only
    ssh-keygen -R ${CL_HOSTNAME}
    # Cluster FQDN
    # ssh-keygen -R ${CL_HOSTNAME}.${DOMAIN}
    # IP Cluster 
    ssh-keygen -R $(getent hosts ${CL_HOSTNAME} | awk '{ print $1 }')
    # Add ip into known_hosts
    ssh-keyscan -H ${PRE_FIX}${i} >> ~/.ssh/known_hosts
    sshpass -p raspberry ssh-copy-id pi@${PRE_FIX}${i}
done

Alle Clients als Worker dem Swarm beitreten.

In [ ]:
for i in 01 02 03 04; do
    CL_HOSTNAME=cluster-${i}
    echo $CL_HOSTNAME
    CL_IP=$(getent hosts ${CL_HOSTNAME} | awk '{ print $1 }')
    echo $CL_IP
    ssh pi@cluster-${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP}:2377"
done

In [ ]:
for i in 21.76 23.27 23.237 23.242; do
    PRE_FIX=172.16.
    CL_HOSTNAME=${PRE_FIX}${i}
    echo $CL_HOSTNAME
    CL_IP=$(getent hosts ${CL_HOSTNAME} | awk '{ print $1 }')
    echo $CL_IP
    ssh pi@${PRE_FIX}${i} "docker swarm join --token $JOIN_TOKEN --advertise-addr ${CL_IP} ${IP}:2377"
done

In [ ]:
docker node ls

## Visualizer anschauen

Einen graphischen Output als Service starten.

In [ ]:
docker service create \
  --name=visualizer \
  --publish=8000:8080/tcp \
  --constraint=node.role==manager \
  --mount=type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  alexellis2/visualizer-arm

In [ ]:
echo "http://$(getent hosts cluster-00.fritz.box | awk '{ print $1 }'):8000"

In [ ]:
echo "http://${IP}:8000"

In [ ]:
docker service ls

Auf allen nodes den Monitor Dienst installieren. Dieser überwacht, wie viele whoami Container laufen.

Grün: Ein whoami-Container wird gestartet.

Rot: Ein whoami-Container stoppt.

Gelb: Für die Version 1.1.0

Blau: Für die Version 1.2.0

In [ ]:
docker service create --name monitor --mode global \
  --restart-condition any --mount type=bind,src=/sys,dst=/sys \
  --mount type=bind,src=/var/run/docker.sock,dst=/var/run/docker.sock \
  stefanscherer/monitor:1.1.0

Es läuft noch kein whoami.

In [ ]:
docker service ls

In [ ]:
docker service create --name whoami stefanscherer/whoami:1.1.0

In [ ]:
sleep 10

In [ ]:
docker service ls

In [ ]:
docker service scale whoami=5

In [ ]:
sleep 30

In [ ]:
docker service ls

In [1]:
docker service scale whoami=15

whoami scaled to 15


In [ ]:
sleep 30

In [ ]:
docker service ls

In [ ]:
docker service scale whoami=40

In [ ]:
sleep 45

In [ ]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

In [ ]:
sleep 45

In [ ]:
docker service ls

In [ ]:
docker service scale whoami=1

In [ ]:
sleep 15

In [ ]:
docker service scale whoami=40

In [ ]:
sleep 45

In [ ]:
docker service ls

In [ ]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.1.0 whoami

In [ ]:
sleep 45

In [ ]:
docker service update --update-parallelism 5 \
  --image stefanscherer/whoami:1.2.0 whoami

In [ ]:
sleep 45

In [ ]:
docker service ls

In [ ]:
END=10
for i in $(seq 1 $END); do
    docker service scale whoami=1 
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 5
    docker service scale whoami=15
    sleep 10
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 15
    docker service scale whoami=40
    sleep 15
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.1.0 whoami --detach=false
    sleep 30
    docker service update --update-parallelism 5 \
        --image stefanscherer/whoami:1.2.0 whoami --detach=false
    sleep 30
done

# Aufräumen

In [ ]:
docker service rm whoami

In [ ]:
docker service rm monitor

In [ ]:
docker node ls

In [ ]:
docker service ls

In [ ]:
for i in 01 02 03 04; do
    ssh pi@cluster-${i} "docker swarm leave --force"
done

In [ ]:
for i in 21.76 23.27 23.237 23.242; do
    PRE_FIX=172.16.
    CL_HOSTNAME=${PRE_FIX}${i}
    ssh pi@${PRE_FIX}${i} "docker swarm leave --force"
done

In [ ]:
docker service rm visualizer

In [ ]:
docker swarm leave --force